In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import tkinter as tk



In [24]:
from database.BuildingRepository import BuildingRepository
from modules import ACStatusAdapter
from datetime import datetime, timedelta
def get_iot_readings():
    building_repo = BuildingRepository()
    iots = building_repo.get_historic_iots(datetime.now() - timedelta(days=1))
    iot_readings_historic = pd.DataFrame(iots)
    iot_readings_historic = iot_readings_historic.drop("_id", axis=1)
    return iot_readings_historic
iot_readings_historic = get_iot_readings()
iot_readings_historic

,iots,datetime
0,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-17 16:35:55.175
1,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-17 16:35:50.273
2,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-17 16:35:45.205
3,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-17 16:35:40.309
4,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-17 16:35:35.314
...,...,...
17261,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-16 16:36:33.370
17262,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-16 16:36:28.318
17263,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-16 16:36:23.167
17264,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2024-04-16 16:36:18.443


In [25]:
considered_iots = ["Lamp 1_102","Lamp 2_102","Weather","Temperature Sensor 102","Humidity Sensor 102","Light Sensor 102","Air Conditioner 102"]
aux = pd.DataFrame()
for i, row in iot_readings_historic.iterrows():
    new = pd.DataFrame()
    for iot in row['iots']:
        if iot['name'] in considered_iots:
            for value in iot['values']:
                val = value['values']
                new[iot['name'] + '_' + value['type']] = [val]

    aux = pd.concat([aux, new])
    
aux.rename(
    columns={"Weather" + "_temperature": 'Outside temperature (ºC)',
               "Temperature Sensor 102" + "_temperature": 'Temperature (Cº)',
                "Humidity Sensor 102" + "_humidity": 'Humidity (%)',
                "Light Sensor 102" + "_light": 'Light (%)',
                "Lamp 1_102_power": 'Lamp 1',
                "Lamp 2_102_power": 'Lamp 2'},
    inplace=True)
val = aux.values.tolist()
aux = pd.DataFrame(val,columns=['Air Conditioner_power','Air Conditioner_voltage','Air Conditioner_current','Lamp 1','Lamp 2','Temperature (Cº)','Humidity (%)','Light (%)','Outside temperature (ºC)'])
aux

,Air Conditioner 102_power,Air Conditioner 102_voltage,Air Conditioner 102_current,Lamp 1,Lamp 2,Temperature (Cº),Humidity (%),Light (%),Outside temperature (ºC)
0,370.0,224.0,18.0,0.0,80.0,298.0,0.0,100.0,25.8
1,370.0,220.0,18.0,0.0,82.5,296.0,0.0,100.0,25.8
2,370.0,224.0,18.0,0.0,82.5,295.0,0.0,100.0,25.8
3,370.0,224.0,18.0,0.0,82.5,295.0,0.0,100.0,25.1
4,370.0,223.0,18.0,0.0,82.5,296.0,0.0,100.0,25.1
...,...,...,...,...,...,...,...,...,...
17261,0.0,223.0,2.0,0.0,107.5,321.0,0.0,100.0,25.6
17262,0.0,223.0,2.0,0.0,107.5,321.0,0.0,100.0,25.6
17263,0.0,221.0,2.0,0.0,110.0,321.0,0.0,100.0,25.6
17264,0.0,223.0,1.0,0.0,135.0,321.0,0.0,100.0,25.8


In [28]:
df = aux
df['AC status'] = df.apply(lambda x: 0 if x['Air Conditioner 102_power'] == 0 else 1, axis=1)
df['Outside temperature (ºC)'] = df.apply(lambda x: x['Outside temperature (ºC)']*10, axis=1)
df['Occupation'] = df.apply(lambda x: 0 if x['Lamp 1'] and x['Lamp 2'] == 0 else 1, axis=1)
df = df.drop(["Air Conditioner 102_power","Air Conditioner 102_voltage","Air Conditioner 102_current","Lamp 1","Lamp 2"], axis=1)
df['Heat Index (ºC)'] = ACStatusAdapter.calculate_heat_index_custom_celsius(df['Temperature (Cº)'], df['Humidity (%)'])
df

,Temperature (Cº),Humidity (%),Light (%),Outside temperature (ºC),AC status,Occupation,Heat Index (ºC)
0,298.0,0.0,100.0,258.0,1,1,-621.88424
1,296.0,0.0,100.0,258.0,1,1,-610.48196
2,295.0,0.0,100.0,258.0,1,1,-604.81775
3,295.0,0.0,100.0,251.0,1,1,-604.81775
4,296.0,0.0,100.0,251.0,1,1,-610.48196
...,...,...,...,...,...,...,...
17261,321.0,0.0,100.0,256.0,0,1,-760.08871
17262,321.0,0.0,100.0,256.0,0,1,-760.08871
17263,321.0,0.0,100.0,256.0,0,1,-760.08871
17264,321.0,0.0,100.0,258.0,0,1,-760.08871


In [38]:
#Oversampling 
X = df[['Outside temperature (ºC)', 'Temperature (Cº)', 'Humidity (%)', 'Heat Index (ºC)', 'Occupation']]
y = df['AC status']
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
print("Tamanho base de dados após oversampling:", len(X_resampled))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.19, random_state=42)
#Grid Search
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)

grid_search.fit(X_resampled, y_resampled)

print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Melhor pontuação de validação cruzada:", grid_search.best_score_)

Tamanho base de dados após oversampling: 31752


Melhores hiperparâmetros: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5}
Melhor pontuação de validação cruzada: 0.8895270110603203


In [40]:
best_clf = grid_search.best_estimator_

#Treinar o modelo
best_clf.fit(X_resampled, y_resampled)

#Avaliação do modelo
y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy * 100:.5f}%')

Acurácia do modelo: 99.32040%


In [43]:
y_pred

array([0, 1, 0, ..., 0, 1, 0])

In [41]:
import pickle

# Save the model to a file
model_filename = './modules/ACStatus/grid_search.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_clf, model_file)

In [ ]:
#Carregar dados
data_set = pd.read_excel('C:\\Users\\Bruno Ribeiro\\Desktop\\tese\\Bases de dados\\Sala 103\\Base_dados_103_clean.xlsx')
data_set.head()

In [ ]:
print("Tamanho base de dados original:", len(data_set))

In [ ]:
#Oversampling 
X = data_set[['Outside temperature (x10)(ºC)', 'Temperature 103 (x10)(ºC)', 'Humidity 103 (x10)(%)', 'Heat Index 103 (x10)(ºC)', 'Occupation ']]
y = data_set['AC status 103 (bool)']
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
#Tamanho base de dados após oversampling
print("Tamanho base de dados após oversampling:", len(X_resampled))

In [ ]:
#Divisão dados treino + teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.19, random_state=42)

In [ ]:
#Grid Search
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)

grid_search.fit(X_resampled, y_resampled)

print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Melhor pontuação de validação cruzada:", grid_search.best_score_)

In [ ]:
#Treinar modelo 

#Treinar o modelo com os hiperparâmetros encontrados pelo Grid Search
best_clf = grid_search.best_estimator_

#Treinar o modelo
best_clf.fit(X_resampled, y_resampled)

#Avaliação do modelo
y_pred = best_clf.predict(X_test)


In [ ]:
#acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy * 100:.5f}%')

In [ ]:
#Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(cm)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
class_labels = ['-1', '0', '1']

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.show()


In [ ]:
#Precisão, Recall e F1-Score

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Precisão: {precision:.10f}')
print(f'Recall: {recall:.10f}')
print(f'F1-score: {f1:.10f}')

In [ ]:
#validação cruzada (k-fold com k=5)
scores = cross_val_score(best_clf, X_resampled, y_resampled, cv=5)

print("Pontuações da validação cruzada:", scores)

In [ ]:
#média e o desvio padrão das pontuações da validação cruzada
print(f'Média das pontuações: {scores.mean():.10f}')
print(f'Desvio padrão das pontuações: {scores.std():.10f}')

In [ ]:
#Previsões com base nos valores de entrada
def predict_ac_status():
    outside_temp = float(entry_outside_temp.get())
    temp_103 = float(entry_temp_103.get())
    humidity_103 = float(entry_humidity_103.get())
    heat_index_103 = float(entry_heat_index_103.get())
    occupation = float(entry_occupation.get())

    #Previsão com base nos valores inseridos
    prediction = best_clf.predict([[outside_temp, temp_103, humidity_103, heat_index_103, occupation]])

    # Controlar o resultado para "Ligar AC em quente", "Ligar AC em frio" ou "desligar o AC"
    if prediction[0] == -1:
        result_label.config(text="Ligar AC em quente")
    elif prediction[0] == 1:
        result_label.config(text="Ligar AC em frio")
    else:
        result_label.config(text="Desligar o AC")

In [ ]:
#Interface
root = tk.Tk()
root.title("Sistema de Controlo do AC")

frame = tk.Frame(root)
frame.pack(padx=20, pady=20)

#Entrada dados
entry_outside_temp = tk.Entry(frame, width=10, justify='center')
entry_temp_103 = tk.Entry(frame, width=10, justify='center')
entry_humidity_103 = tk.Entry(frame, width=10, justify='center')
entry_heat_index_103 = tk.Entry(frame, width=10, justify='center')
entry_occupation = tk.Entry(frame, width=10, justify='center')

entry_outside_temp.grid(row=0, column=1)
entry_temp_103.grid(row=1, column=1)
entry_humidity_103.grid(row=2, column=1)
entry_heat_index_103.grid(row=3, column=1)
entry_occupation.grid(row=4, column=1)

#Título das variáveis
tk.Label(frame, text="Outside Temperature (x10)(ºC)").grid(row=0, column=0)
tk.Label(frame, text="Temperature 103 (x10)(ºC)").grid(row=1, column=0)
tk.Label(frame, text="Humidity (x10)(%)").grid(row=2, column=0)
tk.Label(frame, text="Heat Index 103 (x10)(ºC)").grid(row=3, column=0)
tk.Label(frame, text="Occupation").grid(row=4, column=0)

#Botão 'Prever'
predict_button = tk.Button(frame, text="Prever", command=predict_ac_status)
predict_button.grid(row=5, columnspan=2)

#Resultado
result_label = tk.Label(frame, text="")
result_label.grid(row=6, columnspan=2)

root.mainloop()